# EXTRAGAL vs STAR



In [1]:
%matplotlib inline
# from sklearn.ensemble import RandomForestClassifier
import optuna
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)
from astropy.table import Table
from astropy.io import fits

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    f1_score, 
    accuracy_score, 
    precision_score,
    recall_score,
    matthews_corrcoef,
    )


# def make_binary_classification_target(y, pos_label, verbose=False):
#     '''Turn multi-class targets into binary classification targets.'''
#     pos_idx = (y==pos_label)
#     y[pos_idx] = 1
#     y[~pos_idx] = 0
#     if verbose:
#         print ('Positive target:\t{}'.format(pos_label))
#         print ('Imbalance ratio:\t{:.3f}'.format((y==0).sum()/(y==1).sum()))
#     y = y.astype(int)
#     return y

In [2]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

# Load extragal&star binary data

In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(["EXTRAGAL","STAR"])
df = pd.read_csv('trainingset_binary_test6.csv')
features = ['iz','zy','yj','jh','hk','iw1','zw1','yw1','jw1','hw1','kw1','w1w2']

In [4]:
X = df[features].values
y = le.transform(df['class'])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=8888)

In [6]:
X_train.shape

(567156, 12)

In [7]:
y_train.shape

(567156,)

In [8]:
y_train[np.where(y_train==0)].shape, y_train[np.where(y_train==1)].shape

((186026,), (381130,))

In [9]:
y_test.shape

(141789,)

# Train model using XGBoost CV

In [10]:
def objective(trial):
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    param = {
        'verbosity': 1,
#         'objective': 'multi:softprob',
#         'num_class':2, 
         'objective': 'binary:logistic',  # 注意objective不是multi
        'booster': 'gbtree',
        'tree_method': 'hist',
        'lambda': trial.suggest_uniform('lambda', 0.5, 3.0),
        'alpha': trial.suggest_uniform('alpha', 0, 2.0),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'eta': trial.suggest_categorical('eta', [0.01, 0.1, 0.2, 0.3]), #learning_rate
        'gamma': trial.suggest_uniform('gamma', 0, 3.0), #min_split_loss
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'subsample': trial.suggest_uniform('subsample', 0.8, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.8, 1.0),
        'max_delta_step': trial.suggest_int('max_delta_step', 1, 8),
#         'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1.0, 100.0),
#         'num_round': int(trial.suggest_discrete_uniform('num_round', 100, 2000, 100)) 
    }

#     bst = xgb.train(param, dtrain, num_boost_round=param['num_round'])
#     preds = bst.predict(dtest)
#     pred_labels = np.rint(preds)
    
#     accuracy = accuracy_score(y_test, pred_labels)
#     precision = precision_score(y_true=y_test, y_pred=pred_labels)
#     recall = recall_score(y_true=y_test, y_pred=pred_labels)
#     f1 = f1_score(y_true=y_test, y_pred=pred_labels)
#     matcoef = matthews_corrcoef(y_test, pred_labels)
#     trial.set_user_attr('accuracy', accuracy)
#     trial.set_user_attr('precision', precision)
#     trial.set_user_attr('recall', recall)
#     trial.set_user_attr('f1', f1)
#     trial.set_user_attr('matcoef', matcoef)
    
#     early_stop = int(param['num_round']/10)


    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'test-logloss')
    history = xgb.cv(
        param,
        dtrain,
        num_boost_round=100,  # fixed,because the increasing num boost round can offset the decreasing eta
        nfold=5, 
        metrics=['logloss'],
        early_stopping_rounds=10, 
        stratified=True, 
        seed=8888,
        callbacks=[pruning_callback]
    )
    mean_logloss = history['test-logloss-mean'].values[-1]
    return mean_logloss

In [11]:
if __name__ == '__main__':
    pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)
    study = optuna.create_study(pruner=pruner, direction='minimize')
    study.optimize(objective, 
                   #n_trials=50
                   n_trials=500
                  )


[I 2023-05-10 14:43:49,125] A new study created in memory with name: no-name-8e78f195-c93b-4cb5-9aa9-186709caf5d2
[I 2023-05-10 14:44:03,788] Trial 0 finished with value: 0.0030614 and parameters: {'lambda': 1.4141112966351086, 'alpha': 1.2121364191923663, 'max_depth': 9, 'eta': 0.1, 'gamma': 1.2781633842873337, 'grow_policy': 'lossguide', 'min_child_weight': 4, 'subsample': 0.9010956829022834, 'colsample_bytree': 0.8580494536660824, 'max_delta_step': 5}. Best is trial 0 with value: 0.0030614.
[I 2023-05-10 14:44:15,671] Trial 1 finished with value: 0.0038129999999999995 and parameters: {'lambda': 2.4702951731161815, 'alpha': 1.5802412466382503, 'max_depth': 4, 'eta': 0.1, 'gamma': 1.5286298319177045, 'grow_policy': 'lossguide', 'min_child_weight': 2, 'subsample': 0.8247011091766142, 'colsample_bytree': 0.8821693439761107, 'max_delta_step': 6}. Best is trial 0 with value: 0.0030614.
[I 2023-05-10 14:44:31,781] Trial 2 finished with value: 0.0030356000000000003 and parameters: {'lambda'

[I 2023-05-10 14:48:00,907] Trial 26 pruned. Trial was pruned at iteration 39.
[I 2023-05-10 14:48:03,910] Trial 27 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 14:48:14,400] Trial 28 finished with value: 0.0030322 and parameters: {'lambda': 1.2108964067902186, 'alpha': 0.11125643949587571, 'max_depth': 6, 'eta': 0.3, 'gamma': 1.897941986470308, 'grow_policy': 'lossguide', 'min_child_weight': 1, 'subsample': 0.8151128783709182, 'colsample_bytree': 0.898912658794999, 'max_delta_step': 5}. Best is trial 10 with value: 0.0029186.
[I 2023-05-10 14:48:17,951] Trial 29 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 14:48:21,331] Trial 30 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 14:48:31,224] Trial 31 finished with value: 0.0029794 and parameters: {'lambda': 2.142686894035019, 'alpha': 0.12582653417833908, 'max_depth': 6, 'eta': 0.3, 'gamma': 0.5565158715799942, 'grow_policy': 'lossguide', 'min_child_weight': 2, 'subsample': 0.8531903467366837, 'colsample_by

[I 2023-05-10 14:53:58,134] Trial 75 pruned. Trial was pruned at iteration 69.
[I 2023-05-10 14:54:01,624] Trial 76 pruned. Trial was pruned at iteration 16.
[I 2023-05-10 14:54:04,398] Trial 77 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 14:54:13,873] Trial 78 finished with value: 0.0029370000000000004 and parameters: {'lambda': 2.987044070111179, 'alpha': 1.2908414792145713, 'max_depth': 8, 'eta': 0.3, 'gamma': 1.181168581348354, 'grow_policy': 'depthwise', 'min_child_weight': 4, 'subsample': 0.9131446131120018, 'colsample_bytree': 0.964500408233102, 'max_delta_step': 7}. Best is trial 65 with value: 0.0028516.
[I 2023-05-10 14:54:18,049] Trial 79 pruned. Trial was pruned at iteration 25.
[I 2023-05-10 14:54:26,952] Trial 80 finished with value: 0.002924 and parameters: {'lambda': 2.8046637758050577, 'alpha': 1.1772347188233356, 'max_depth': 8, 'eta': 0.3, 'gamma': 1.017381644102371, 'grow_policy': 'depthwise', 'min_child_weight': 4, 'subsample': 0.9342774876488196, 'cols

[I 2023-05-10 14:58:09,316] Trial 114 finished with value: 0.0029110000000000004 and parameters: {'lambda': 2.9981562193759146, 'alpha': 1.2762410119459722, 'max_depth': 9, 'eta': 0.3, 'gamma': 0.8344421926455992, 'grow_policy': 'depthwise', 'min_child_weight': 3, 'subsample': 0.9147028133955797, 'colsample_bytree': 0.979714136993964, 'max_delta_step': 7}. Best is trial 92 with value: 0.002846.
[I 2023-05-10 14:58:11,803] Trial 115 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 14:58:14,701] Trial 116 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 14:58:22,078] Trial 117 pruned. Trial was pruned at iteration 65.
[I 2023-05-10 14:58:32,562] Trial 118 finished with value: 0.0029051999999999997 and parameters: {'lambda': 2.5667771165422613, 'alpha': 1.5019811313124718, 'max_depth': 9, 'eta': 0.3, 'gamma': 1.2866492482720688, 'grow_policy': 'depthwise', 'min_child_weight': 4, 'subsample': 0.9007114489189241, 'colsample_bytree': 0.9907672299065217, 'max_delta_step': 7}. Be

[I 2023-05-10 15:02:14,064] Trial 163 finished with value: 0.0028826 and parameters: {'lambda': 2.642293769482375, 'alpha': 1.165619464647854, 'max_depth': 9, 'eta': 0.3, 'gamma': 0.9167409319178494, 'grow_policy': 'depthwise', 'min_child_weight': 4, 'subsample': 0.9172079954356833, 'colsample_bytree': 0.9652806005553888, 'max_delta_step': 7}. Best is trial 92 with value: 0.002846.
[I 2023-05-10 15:02:23,110] Trial 164 finished with value: 0.0029104 and parameters: {'lambda': 2.6109959341704894, 'alpha': 1.1566818313747553, 'max_depth': 9, 'eta': 0.3, 'gamma': 0.907404916197667, 'grow_policy': 'depthwise', 'min_child_weight': 4, 'subsample': 0.9172266111149433, 'colsample_bytree': 0.9674735031771199, 'max_delta_step': 8}. Best is trial 92 with value: 0.002846.
[I 2023-05-10 15:02:27,688] Trial 165 pruned. Trial was pruned at iteration 30.
[I 2023-05-10 15:02:36,171] Trial 166 finished with value: 0.0029012 and parameters: {'lambda': 2.6665926513644416, 'alpha': 1.2184198713648908, 'max

[I 2023-05-10 15:06:24,174] Trial 212 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:06:27,152] Trial 213 pruned. Trial was pruned at iteration 16.
[I 2023-05-10 15:06:29,656] Trial 214 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:06:32,349] Trial 215 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:06:39,532] Trial 216 pruned. Trial was pruned at iteration 70.
[I 2023-05-10 15:06:46,590] Trial 217 pruned. Trial was pruned at iteration 52.
[I 2023-05-10 15:06:49,285] Trial 218 pruned. Trial was pruned at iteration 12.
[I 2023-05-10 15:06:52,099] Trial 219 pruned. Trial was pruned at iteration 14.
[I 2023-05-10 15:06:54,655] Trial 220 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:06:57,100] Trial 221 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:07:01,762] Trial 222 pruned. Trial was pruned at iteration 35.
[I 2023-05-10 15:07:11,432] Trial 223 finished with value: 0.0028813999999999997 and parameters: {'lambda': 2.7733286393

[I 2023-05-10 15:10:54,994] Trial 274 finished with value: 0.0028880000000000004 and parameters: {'lambda': 2.8035733880290086, 'alpha': 1.322869543152799, 'max_depth': 9, 'eta': 0.3, 'gamma': 1.115403868710606, 'grow_policy': 'depthwise', 'min_child_weight': 4, 'subsample': 0.9308374546994149, 'colsample_bytree': 0.9790921299960581, 'max_delta_step': 6}. Best is trial 92 with value: 0.002846.
[I 2023-05-10 15:10:58,547] Trial 275 pruned. Trial was pruned at iteration 25.
[I 2023-05-10 15:11:01,500] Trial 276 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:11:04,099] Trial 277 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:11:06,776] Trial 278 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:11:09,607] Trial 279 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:11:12,382] Trial 280 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:11:15,430] Trial 281 pruned. Trial was pruned at iteration 13.
[I 2023-05-10 15:11:19,295] Trial 282 prune

[I 2023-05-10 15:15:17,834] Trial 352 finished with value: 0.0029066 and parameters: {'lambda': 2.614260432778873, 'alpha': 1.2596973351972374, 'max_depth': 9, 'eta': 0.3, 'gamma': 1.2186190800792875, 'grow_policy': 'depthwise', 'min_child_weight': 3, 'subsample': 0.9115417546880136, 'colsample_bytree': 0.9705532773051944, 'max_delta_step': 4}. Best is trial 92 with value: 0.002846.
[I 2023-05-10 15:15:20,749] Trial 353 pruned. Trial was pruned at iteration 15.
[I 2023-05-10 15:15:26,917] Trial 354 pruned. Trial was pruned at iteration 49.
[I 2023-05-10 15:15:29,554] Trial 355 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:15:32,077] Trial 356 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:15:36,083] Trial 357 pruned. Trial was pruned at iteration 28.
[I 2023-05-10 15:15:38,872] Trial 358 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:15:43,415] Trial 359 pruned. Trial was pruned at iteration 32.
[I 2023-05-10 15:15:46,585] Trial 360 pruned. Trial wa

[I 2023-05-10 15:19:27,446] Trial 414 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:19:32,001] Trial 415 pruned. Trial was pruned at iteration 30.
[I 2023-05-10 15:19:34,760] Trial 416 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:19:43,659] Trial 417 finished with value: 0.002884 and parameters: {'lambda': 2.5978351407473523, 'alpha': 1.7758664153415937, 'max_depth': 9, 'eta': 0.3, 'gamma': 1.3625742479109373, 'grow_policy': 'depthwise', 'min_child_weight': 1, 'subsample': 0.919194121223743, 'colsample_bytree': 0.9969483438618009, 'max_delta_step': 6}. Best is trial 92 with value: 0.002846.
[I 2023-05-10 15:19:47,509] Trial 418 pruned. Trial was pruned at iteration 26.
[I 2023-05-10 15:19:52,671] Trial 419 pruned. Trial was pruned at iteration 37.
[I 2023-05-10 15:19:57,436] Trial 420 pruned. Trial was pruned at iteration 35.
[I 2023-05-10 15:20:01,582] Trial 421 pruned. Trial was pruned at iteration 27.
[I 2023-05-10 15:20:04,101] Trial 422 pruned. Trial was

[I 2023-05-10 15:23:45,628] Trial 489 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:23:48,781] Trial 490 pruned. Trial was pruned at iteration 15.
[I 2023-05-10 15:23:53,073] Trial 491 pruned. Trial was pruned at iteration 23.
[I 2023-05-10 15:23:55,795] Trial 492 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:23:58,489] Trial 493 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:24:01,936] Trial 494 pruned. Trial was pruned at iteration 20.
[I 2023-05-10 15:24:06,723] Trial 495 pruned. Trial was pruned at iteration 39.
[I 2023-05-10 15:24:09,416] Trial 496 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:24:12,924] Trial 497 pruned. Trial was pruned at iteration 21.
[I 2023-05-10 15:24:15,888] Trial 498 pruned. Trial was pruned at iteration 10.
[I 2023-05-10 15:24:18,501] Trial 499 pruned. Trial was pruned at iteration 10.


In [12]:
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 500
Best trial:
  Value: 0.002846
  Params: 
    lambda: 2.9077613895719
    alpha: 1.5263290780459133
    max_depth: 9
    eta: 0.3
    gamma: 1.0003397864024768
    grow_policy: depthwise
    min_child_weight: 4
    subsample: 0.9129317176922225
    colsample_bytree: 0.9886871929591262
    max_delta_step: 6


#### n_trials=50 test1
Params: 
    lambda: 2.652579588756935
    alpha: 1.4208405720586856
    max_depth: 8
    eta: 0.3
    gamma: 1.0809389255690658
    grow_policy: lossguide
    min_child_weight: 2
    subsample: 0.9033901196294835
    colsample_bytree: 0.8010648180780473
    max_delta_step: 8
    
#### n_trials=500  test1
Params: 
    lambda: 1.7300539625554947
    alpha: 1.8577287659851582
    max_depth: 6
    eta: 0.3
    gamma: 0.415768753457072
    grow_policy: depthwise
    min_child_weight: 1
    subsample: 0.8027491323812233
    colsample_bytree: 0.8474931094456103
    max_delta_step: 4
    

#### n_trials=500  test2
Params: 
    lambda: 2.1166099315127616
    alpha: 0.37588851885081564
    max_depth: 9
    eta: 0.2
    gamma: 0.2517811355164694
    grow_policy: lossguide
    min_child_weight: 1
    subsample: 0.9604551106992315
    colsample_bytree: 0.8664726082942656
    max_delta_step: 5
  
  
#### n_trials=500 test3
Params: 
    lambda: 2.704201450052538
    alpha: 1.9039335520826655
    max_depth: 9
    eta: 0.3
    gamma: 0.806966094369884
    grow_policy: depthwise
    min_child_weight: 2
    subsample: 0.999660527143889
    colsample_bytree: 0.8946854701422615
    max_delta_step: 3
    
    
#### n_trials=500 test4
Params: 
    lambda: 2.3158232520886988
    alpha: 1.7542852705839933
    max_depth: 8
    eta: 0.2
    gamma: 0.49828185695738614
    grow_policy: depthwise
    min_child_weight: 1
    subsample: 0.9127692593908899
    colsample_bytree: 0.835862726945685
    max_delta_step: 4

#### n_trials=500 test5
Params: 
    lambda: 1.3803592642929852
    alpha: 1.7891090581748477
    max_depth: 9
    eta: 0.3
    gamma: 1.1302960282498806
    grow_policy: lossguide
    min_child_weight: 2
    subsample: 0.9077021064990229
    colsample_bytree: 0.8718345678647781
    max_delta_step: 4
    
    
#### n_trials=500 test6
Params: 
    lambda: 2.9077613895719
    alpha: 1.5263290780459133
    max_depth: 9
    eta: 0.3
    gamma: 1.0003397864024768
    grow_policy: depthwise
    min_child_weight: 4
    subsample: 0.9129317176922225
    colsample_bytree: 0.9886871929591262
    max_delta_step: 6

In [13]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_eta,params_gamma,params_grow_policy,params_lambda,params_max_delta_step,params_max_depth,params_min_child_weight,params_subsample,state
0,0,0.003061,2023-05-10 14:43:49.127781,2023-05-10 14:44:03.787057,0 days 00:00:14.659276,1.212136,0.858049,0.10,1.278163,lossguide,1.414111,5,9,4,0.901096,COMPLETE
1,1,0.003813,2023-05-10 14:44:03.790921,2023-05-10 14:44:15.669817,0 days 00:00:11.878896,1.580241,0.882169,0.10,1.528630,lossguide,2.470295,6,4,2,0.824701,COMPLETE
2,2,0.003036,2023-05-10 14:44:15.675826,2023-05-10 14:44:31.779861,0 days 00:00:16.104035,0.713466,0.985197,0.10,0.721599,lossguide,2.625473,6,8,1,0.872493,COMPLETE
3,3,0.206084,2023-05-10 14:44:31.783694,2023-05-10 14:44:40.752406,0 days 00:00:08.968712,1.955230,0.914379,0.01,0.524250,depthwise,1.724083,8,5,4,0.926808,COMPLETE
4,4,0.205163,2023-05-10 14:44:40.758344,2023-05-10 14:44:55.540758,0 days 00:00:14.782414,0.620246,0.882312,0.01,0.690640,lossguide,0.621643,8,9,5,0.925318,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,0.003024,2023-05-10 15:24:01.940541,2023-05-10 15:24:06.723377,0 days 00:00:04.782836,1.174434,0.959639,0.30,1.227537,depthwise,2.199987,7,9,3,0.888942,PRUNED
496,496,0.020140,2023-05-10 15:24:06.728475,2023-05-10 15:24:09.416198,0 days 00:00:02.687723,1.127146,0.968621,0.30,1.390579,depthwise,2.507998,7,9,4,0.906494,PRUNED
497,497,0.003906,2023-05-10 15:24:09.420036,2023-05-10 15:24:12.924520,0 days 00:00:03.504484,1.191532,0.996052,0.30,1.269874,depthwise,2.580631,7,9,4,0.986652,PRUNED
498,498,0.020195,2023-05-10 15:24:12.929291,2023-05-10 15:24:15.888295,0 days 00:00:02.959004,0.956616,0.893787,0.30,1.350410,depthwise,2.667813,7,8,4,0.882932,PRUNED


In [14]:
study.best_trial

FrozenTrial(number=92, values=[0.002846], datetime_start=datetime.datetime(2023, 5, 10, 14, 55, 37, 270265), datetime_complete=datetime.datetime(2023, 5, 10, 14, 55, 48, 775667), params={'lambda': 2.9077613895719, 'alpha': 1.5263290780459133, 'max_depth': 9, 'eta': 0.3, 'gamma': 1.0003397864024768, 'grow_policy': 'depthwise', 'min_child_weight': 4, 'subsample': 0.9129317176922225, 'colsample_bytree': 0.9886871929591262, 'max_delta_step': 6}, distributions={'lambda': UniformDistribution(high=3.0, low=0.5), 'alpha': UniformDistribution(high=2.0, low=0.0), 'max_depth': IntUniformDistribution(high=9, low=3, step=1), 'eta': CategoricalDistribution(choices=(0.01, 0.1, 0.2, 0.3)), 'gamma': UniformDistribution(high=3.0, low=0.0), 'grow_policy': CategoricalDistribution(choices=('depthwise', 'lossguide')), 'min_child_weight': IntUniformDistribution(high=5, low=1, step=1), 'subsample': UniformDistribution(high=1.0, low=0.8), 'colsample_bytree': UniformDistribution(high=1.0, low=0.8), 'max_delta_s

In [15]:
opdf = study.trials_dataframe()
from pathlib import Path
Path("param_tune_records").mkdir(exist_ok=True)
opdf.to_pickle("param_tune_records/optuna_xgb_ext_vs_star_cv-logloss-500trials-6.pkl")